download the kitti dataset

In [51]:
from torchvision.datasets import Kitti
import torchvision
from torch.utils.data import DataLoader
import torch
import torch.nn as nn
import torch.optim as optim


In [52]:


def custom_collate_fn(data):
  return tuple(zip(*data))
data_path = "./train/"
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor(),
    # hint: there is something missin here
    # change the size of the image to 3x224x224
    torchvision.transforms.Resize((224, 224)),

    ])

train_dataset = Kitti(
    data_path,
    train= True, # hint: its a boolean
    transform=transform,
    download=True
    )

test_dataset = Kitti(
    data_path,
    train= False, # hint: its a boolean
    transform=transform,
    download=True
    )

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=custom_collate_fn)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, collate_fn=custom_collate_fn)

In [53]:
kiti_classes = {'Car': 0,
 'Cyclist': 1,
 'DontCare': 2,
 'Misc': 3,
 'Pedestrian': 4,
 'Person_sitting': 5,
 'Tram': 6,
 'Truck': 7,
 'Van': 8}

importing the mobilenet model

In [54]:
# travers all the data to check the number of classes
classes = []
for i in range(len(train_dataset)):
  classes.append(train_dataset[i][1])
# classes = set(classes)
print("Number of classes: ", len(classes))
    

Number of classes:  7481


In [55]:
classes = [cls[0]['type'] for cls in classes]
classes[:4]

['Pedestrian', 'Truck', 'Misc', 'Car']

In [56]:
clset = set(classes)
print("Number of classes: ", len(clset))

Number of classes:  8


In [57]:
# check size of the image
img = train_dataset[0][0]
print("Image size: ", img.size())

Image size:  torch.Size([3, 224, 224])


In [69]:
# mobilenet_v2

In [76]:
from torchvision import models
mobilenet_v2 = models.mobilenet_v2(pretrained=True)


c:\Users\trh00\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\trh00\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [77]:
# freeze all layer except the last one
for param in mobilenet_v2.parameters():
    param.requires_grad = False

# change the last layer to fit our problem
mobilenet_v2.classifier[1] = torch.nn.Linear(1280, 9)


In [78]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mobilenet_v2.to(device)
optimizer = torch.optim.Adam(mobilenet_v2.parameters(), lr=0.001)
num_epochs = 10

In [79]:
from tqdm import tqdm
for epoch in range(num_epochs):
    mobilenet_v2.train()
    for images, targets in tqdm(train_loader):

        images = list(image.to(device) for image in images)
        images = torch.stack(images).to(device)  # Shape: [batch_size, 3, height, width]
        new_targets = []
        for target in targets:
          boxes = []
          labels = []
          for obj in target:
            label = obj['type']
            box = obj['bbox']
            # xmin, ymin, xmax, ymax = [int(box[k]) for k in ['xmin', 'ymin', 'xmax', 'ymax']]
            box = torch.Tensor(box).to(device)
            boxes.append(box)
            labels.append(kiti_classes[label])
          target = {}
          target['boxes'] = torch.stack(boxes)
          target['labels'] = torch.Tensor(labels).type(torch.int64).to(device)
          new_targets.append(target)


        # In training mode, the model will take images and targets and calculate all the losses in internal implementation
        output = mobilenet_v2(images)
        print(output)

        losses = sum(loss for loss in loss_dict.values())  # Sum all the different losses.

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {losses.item()}")

  0%|          | 0/117 [00:01<?, ?it/s]

torch.Size([64, 9])


RuntimeError: values expected sparse tensor layout but got Strided